# RAG
La generación aumentada por recuperación (RAG) es el proceso de optimización de la salida de un modelo de lenguaje de gran tamaño, de modo que haga referencia a una base de conocimientos autorizada fuera de los orígenes de datos de entrenamiento antes de generar una respuesta. 

### Instalación de dependencias:

```bash
uv pip install langchain-community langchain-core langchain-ollama pypdf

```

### Cargar los pdfs

Recibimos el path del documento pdf por parametro. A partir del documento recibido, lo convertimos en un iterable. 
Al final, retornamos la variable text con el contenido del documento crudo, sin dividir. 

In [67]:
from langchain_community.document_loaders import PyPDFLoader

def upload_pdf(url: str):        
    try:
        loader = PyPDFLoader(url)
        loader = loader.lazy_load()

        text = ""

        for page in loader: 
            text += page.page_content + "\n"

        return text
    except Exception as e:
        print(e)
        return []
    


### text_splitter

Recibimos el contenido del documento generado en la celda anterior. Ahora parametrizamos `chunk_size` y `chunk_overlap` para poder experimentar con distintos tamaños de fragmento. Como configuración por defecto reducimos los trozos a 1 200 caracteres con 150 de solapamiento para capturar contexto suficiente sin repetir tanto contenido.


In [68]:
from langchain_text_splitters import CharacterTextSplitter


def text_splitter(
    text: str,
    *,
    chunk_size: int = 1200,
    chunk_overlap: int = 150,
    separator: str = "\n",
    base_metadata: dict | None = None,
    report: bool = True,
) -> list:

    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separator=separator,
    )
    base_metadata = base_metadata or {}
    texts = splitter.create_documents([text], metadatas=[base_metadata])

    fallback_source = base_metadata.get("source", "uploaded_pdf")
    fallback_topic = base_metadata.get("topic", "stuxnet")
    for idx, doc in enumerate(texts):
        doc.metadata = dict(doc.metadata)
        doc.metadata.setdefault("source", fallback_source)
        doc.metadata.setdefault("topic", fallback_topic)
        doc.metadata["chunk_index"] = idx
        normalized = doc.page_content.lower()
        doc.metadata["mentions_plc"] = "plc" in normalized or "centrifug" in normalized
        doc.metadata["mentions_windows"] = "windows" in normalized or "siemens" in normalized

    if report and texts:
        lengths = [len(doc.page_content) for doc in texts]
        avg_len = sum(lengths) / len(lengths)
        print(
            f"Generados {len(texts)} chunks | media: {avg_len:.0f} | min: {min(lengths)} | max: {max(lengths)}"
        )
    elif report:
        print("No se generaron chunks. Ajusta los parámetros del splitter.")

    return texts


### embedding

Creamos la conexión con nuestro modelo de embedding. 

In [69]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model = "nomic-embed-text"
)

### vector store

Creamos nuestra vector store con FAISS para mantener colecciones locales que podamos reconstruir, persistir en disco y filtrar mediante metadatos enriquecidos con el splitter.


In [70]:
from pathlib import Path
from typing import Any, Dict, List, Optional

from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

FAISS_INDEX_DIR = Path("vectorstore/faiss_langchain")


def _faiss_index_exists(directory: Path) -> bool:
    return (directory / "index.faiss").exists() and (directory / "index.pkl").exists()


def load_faiss_vector_store(*, directory: Optional[Path] = None) -> FAISS:
    target_dir = Path(directory) if directory else FAISS_INDEX_DIR
    if not _faiss_index_exists(target_dir):
        raise FileNotFoundError(
            f"No se encontró un índice FAISS en {target_dir}. Ejecuta `upsert_faiss_collection` para generarlo."
        )
    return FAISS.load_local(
        str(target_dir),
        embeddings=embedding,
        allow_dangerous_deserialization=True,  # requerido para deserializar los pickles del índice
    )


def upsert_faiss_collection(
    documents: List[Document],
    *,
    directory: Optional[Path] = None,
    rebuild: bool = False,
) -> FAISS:
    target_dir = Path(directory) if directory else FAISS_INDEX_DIR
    target_dir.mkdir(parents=True, exist_ok=True)
    if rebuild or not _faiss_index_exists(target_dir):
        vector_store = FAISS.from_documents(documents, embedding)
    else:
        vector_store = load_faiss_vector_store(directory=target_dir)
        vector_store.add_documents(documents)
    vector_store.save_local(str(target_dir))
    return vector_store


def faiss_similarity_search(
    query: str,
    *,
    k: int = 4,
    metadata_filter: Optional[Dict[str, Any]] = None,
    score_threshold: Optional[float] = None,
    directory: Optional[Path] = None,
):
    vector_store = load_faiss_vector_store(directory=directory)
    docs_and_scores = vector_store.similarity_search_with_score(
        query,
        k=k,
        filter=metadata_filter,
    )
    if score_threshold is not None:
        docs_and_scores = [
            (doc, score) for doc, score in docs_and_scores if score <= score_threshold
        ]
    return docs_and_scores


#### Gestión práctica del índice FAISS

`index_pdf_into_faiss` levanta el pipeline completo (ingesta + enriquecimiento de metadatos) mientras que `preview_faiss_hits` ayuda a evaluar búsquedas con filtros o umbrales antes de llamar al LLM.


In [71]:
from typing import Optional


def index_pdf_into_faiss(
    pdf_path: str,
    *,
    topic: str = "stuxnet",
    rebuild: bool = False,
):
    raw_text = upload_pdf(pdf_path)
    documents = text_splitter(
        raw_text,
        base_metadata={
            "source": pdf_path,
            "topic": topic,
        },
    )
    return upsert_faiss_collection(documents, rebuild=rebuild)


def preview_faiss_hits(
    question: str,
    *,
    top_k: int = 4,
    filter_plc: bool = False,
    score_threshold: Optional[float] = None,
):
    metadata_filter = {"mentions_plc": True} if filter_plc else None
    docs_and_scores = faiss_similarity_search(
        question,
        k=top_k,
        metadata_filter=metadata_filter,
        score_threshold=score_threshold,
    )
    formatted = []
    for doc, score in docs_and_scores:
        formatted.append(
            {
                "chunk_index": doc.metadata.get("chunk_index"),
                "source": doc.metadata.get("source"),
                "topic": doc.metadata.get("topic"),
                "mentions_plc": doc.metadata.get("mentions_plc"),
                "score": round(score, 4),
                "preview": doc.page_content[:200].strip().replace("\n", " "),
            }
        )
    return formatted


### Retrieval

Creamos una función retriebal, que nos devuelve los documentos en una busqueda de similitudes. 

In [72]:
from typing import Any, Dict, Optional


def retrieval(
    input_user: str,
    *,
    k: int = 4,
    metadata_filter: Optional[Dict[str, Any]] = None,
    score_threshold: Optional[float] = None,
):
    docs_and_scores = faiss_similarity_search(
        input_user,
        k=k,
        metadata_filter=metadata_filter,
        score_threshold=score_threshold,
    )
    return [doc for doc, _ in docs_and_scores]


### Prompt template

Integramos varias plantillas de prompt parametrizables (tono, idioma y reglas de citación) para experimentar con instrucciones de sistema/humano distintas antes de llamar al LLM.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate




baseline_prompt = PromptTemplate.from_template(
    """
Eres un asistente especializado en LangChain. Responde en {answer_language} con un tono {style}.
- Limítate al contexto proporcionado y reconoce con honestidad cuando falten datos.
Contexto verificado:
{contexto}

Pregunta del usuario:
{input_user}

Respuesta:
"""
)

citation_prompt = PromptTemplate.from_template(
    """
Actúas como redactor técnico enfocado en LangChain y debes atribuir cada afirmación.
Reglas:
1. Prioriza la precisión sobre la cobertura y responde en {answer_language} con un tono {style}.
2. Cada frase que use el contexto debe terminar con el marcador [Fuente X] que corresponda al bloque numerado del contexto.
3. Si no existe evidencia en el contexto, declara la limitación.
Contexto numerado:
{contexto}

Pregunta:
{input_user}

Respuesta estructurada:
- Conclusión principal
- Detalles clave
- Riesgos o incógnitas

Fuentes consultadas: lista de [Fuente X]
"""
)

deliberate_chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un analista de ciberseguridad. Responde en {answer_language} con un tono {style}, menciona incertidumbres y cita las fuentes del contexto usando [Fuente X].",
        ),
        (
            "ai",
            "Entendido, validaré la evidencia antes de emitir la respuesta final.",
        ),
        (
            "human",
            "Contexto a citar:\n{contexto}\n\nPregunta del usuario:\n{input_user}\n\n"
            "Plan: 1) Resume la evidencia clave, 2) Menciona lagunas si existen, "
            "3) Entrega la respuesta final breve citando [Fuente X].",
        ),
    ]
)




PROMPT_VARIANTS = {
    "baseline": baseline_prompt,
    "citations": citation_prompt,
    "deliberate_chat": deliberate_chat_prompt,
}




def render_prompt(variant: str = "baseline", **kwargs):
    """
    Renderiza un prompt normal o de chat según su tipo.
    Devuelve:
      - texto (str) si es PromptTemplate
      - lista de mensajes (list[dict]) si es ChatPromptTemplate
    """
    template = PROMPT_VARIANTS.get(variant)

    if template is None:
        raise ValueError(f"Prompt variant desconocido: {variant}")

    
    kwargs.setdefault("answer_language", "es")
    kwargs.setdefault("style", "directa")

  
    if isinstance(template, ChatPromptTemplate):
        return template.format_messages(**kwargs)

    
    return template.format(**kwargs)


#### Estrategias de prompting incluidas

- `baseline`: respuesta directa y honesta enfocada en preguntas frecuentes.
- `citations`: obliga a citar cada afirmación con el marcador `[Fuente X]` generado a partir de los metadatos.
- `deliberate_chat`: usa mensajes de sistema/humano para guiar un razonamiento breve antes de entregar la respuesta final.

`render_prompt` permite alternar dinámicamente entre estas variantes desde cualquier flujo del cuaderno.


In [74]:
from typing import Sequence


def format_docs_with_sources(docs: Sequence[Document], *, max_chars: int = 700) -> str:
    formatted_blocks = []
    for idx, doc in enumerate(docs, start=1):
        metadata = getattr(doc, "metadata", {}) or {}
        source = metadata.get("source", "desconocido")
        chunk = metadata.get("chunk_index", "?")
        topic = metadata.get("topic", "general")
        tags = []
        if metadata.get("mentions_plc"):
            tags.append("PLC")
        if metadata.get("mentions_windows"):
            tags.append("Windows")
        tag_str = ",".join(tags) if tags else "sin-tags"
        snippet = doc.page_content.strip()
        if len(snippet) > max_chars:
            snippet = snippet[:max_chars].rstrip() + "..."
        formatted_blocks.append(
            f"[Fuente {idx} | src={source} | chunk={chunk} | topic={topic} | tags={tag_str}]{snippet}"
        )
    if not formatted_blocks:
        return "No se encontraron fragmentos relevantes para citar."
    return "\n".join(formatted_blocks)


def ensure_context_text(contexto) -> str:
    if isinstance(contexto, str):
        return contexto
    if isinstance(contexto, Sequence):
        return format_docs_with_sources(contexto)
    return str(contexto)


def preview_prompt_variants(
    question: str,
    docs: Sequence[Document],
    *,
    variants: Sequence[str] | None = None,
    answer_language: str = "es",
    style: str = "directa",
):
    context_text = format_docs_with_sources(docs)
    variants = variants or list(PROMPT_VARIANTS.keys())
    previews = {}
    for variant in variants:
        rendered = render_prompt(
            variant,
            contexto=context_text,
            input_user=question,
            answer_language=answer_language,
            style=style,
        )
        if isinstance(rendered, list):
            previews[variant] = "".join(
                f"{message.type.upper()}: {message.content}" for message in rendered
            )
        else:
            previews[variant] = rendered
    return previews


### LLM 

Declaramos la función response, responsable de la interacción con el LLM. 

In [75]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")


def response(
    input_user: str,
    contexto,
    *,
    prompt_variant: str = "baseline",
    answer_language: str = "es",
    style: str = "directa",
):
    llm = ChatGoogleGenerativeAI(
        api_key=api_key,
        model="gemini-2.0-flash",
        temperature=0.5,
    )
    context_text = ensure_context_text(contexto)
    rendered_prompt = render_prompt(
        prompt_variant,
        contexto=context_text,
        input_user=input_user,
        answer_language=answer_language,
        style=style,
    )
    for chunk in llm.stream(rendered_prompt):
        yield chunk.content


Ahora utilizamos las funciones para cargar el documento, aplicar el text_splitter y al final, guardar nuestros datos como embedding en la base de datos vectorial. 

In [76]:
loader = upload_pdf("Stuxnet (1) - copia.pdf")
texts = text_splitter(
    loader,
    base_metadata={
        "source": "Stuxnet (1) - copia.pdf",
        "topic": "stuxnet",
    },
)
faiss_store = upsert_faiss_collection(texts, rebuild=True)


Created a chunk of size 2169, which is longer than the specified 1200


Generados 18 chunks | media: 1180 | min: 272 | max: 2159


## Optimización del Separador de Texto

Modifica los parámetros `chunk_size` y `chunk_overlap` para estudiar cómo cambian los documentos resultantes y la calidad del contexto que llega al LLM. La celda siguiente crea una rutina de experimentación que imprime estadísticas de los chunks, los documentos más parecidos a una pregunta y, opcionalmente, la respuesta producida por el modelo.


In [77]:
import numpy as np
from typing import Dict, List


def _normalize_vector(vector):
    arr = np.array(vector, dtype=float)
    norm = np.linalg.norm(arr)
    return arr if norm == 0 else arr / norm


def chunking_experiments(
    raw_text: str,
    question: str,
    chunk_trials: List[Dict[str, int]],
    top_k: int = 2,
    run_llm: bool = False,
):
    experiment_rows = []

    for trial in chunk_trials:
        docs = text_splitter(
            raw_text,
            chunk_size=trial["chunk_size"],
            chunk_overlap=trial["chunk_overlap"],
            separator=trial.get("separator", "\n"),
            report=False,
        )

        lengths = [len(doc.page_content) for doc in docs]
        stats = {
            "chunks": len(docs),
            "avg_len": int(sum(lengths) / len(lengths)) if lengths else 0,
            "min_len": min(lengths) if lengths else 0,
            "max_len": max(lengths) if lengths else 0,
        }

        doc_texts = [doc.page_content for doc in docs]
        doc_vectors = [
            _normalize_vector(vector)
            for vector in embedding.embed_documents(doc_texts)
        ]
        query_vector = _normalize_vector(embedding.embed_query(question))

        scored_docs = sorted(
            (
                (float(np.dot(query_vector, doc_vector)), doc)
                for doc_vector, doc in zip(doc_vectors, docs)
            ),
            key=lambda item: item[0],
            reverse=True,
        )[:top_k]

        answer = None
        best_docs = [doc for _, doc in scored_docs]
        if run_llm and best_docs:
            context = "\n\n".join(doc.page_content for doc in best_docs)
            answer = "".join(response(input_user=question, contexto=context))

        summary = (
            f"chunk_size={trial['chunk_size']}, overlap={trial['chunk_overlap']} -> "
            f"{stats['chunks']} chunks (media {stats['avg_len']}, "
            f"min {stats['min_len']}, max {stats['max_len']})"
        )
        print(summary)
        for rank, (score, doc) in enumerate(scored_docs, start=1):
            snippet = doc.page_content[:140].replace("\n", " ")
            print(f"  {rank}. score={score:.3f} -> {snippet}...")

        if answer:
            print(f"Respuesta del LLM: {answer.strip()}\n")

        experiment_rows.append(
            {
                "params": trial,
                "stats": stats,
                "top_docs": best_docs,
                "answer": answer,
            }
        )

    return experiment_rows


chunk_trials = [
    {"chunk_size": 800, "chunk_overlap": 200},
    {"chunk_size": 1200, "chunk_overlap": 150},
    {"chunk_size": 1600, "chunk_overlap": 100},
]

sample_question = "¿Cuál es el vector de ataque principal descrito en el informe?"
chunking_summary = chunking_experiments(
    loader, sample_question, chunk_trials, top_k=2, run_llm=False
)


Created a chunk of size 2169, which is longer than the specified 800
Created a chunk of size 2169, which is longer than the specified 1200


chunk_size=800, overlap=200 -> 31 chunks (media 780, min 272, max 2159)
  1. score=0.574 -> computadoras   de   Asia,   por   lo   que   elaboró   la   hipótesis   sobre   que   se   desarrollaron   varios   modelos   para   atacar ...
  2. score=0.562 -> explotar   esta   vulnerabilidad   antes   de   que   se   desarrolle   e   implemente   un   parche   para   solucionarla.   La   falta   d...


Created a chunk of size 2169, which is longer than the specified 1600


chunk_size=1200, overlap=150 -> 18 chunks (media 1179, min 272, max 2159)
  1. score=0.551 -> Seminario  de  Actualización  III:   CiberSeguridad   Exposición:  Casos  Reales  Relacionados  a  la   CiberSeguridad   –   Stuxnet     Gru...
  2. score=0.545 -> a   la   red   local   y   el   gusano   penetró   así   en   el   sistema   de   la   planta   (Falliere,   Murchu   &   Chien,   2011).   ...
chunk_size=1600, overlap=100 -> 14 chunks (media 1433, min 272, max 2159)
  1. score=0.559 -> World's   First   Digital   Weapon .   Crown   Publishing   Group.    ●  BBC  News  Mundo.  (2015,  octubre  7).  Stuxnet:  el  virus  infor...
  2. score=0.551 -> Seminario  de  Actualización  III:   CiberSeguridad   Exposición:  Casos  Reales  Relacionados  a  la   CiberSeguridad   –   Stuxnet     Gru...


### Comparativa de separadores alternativos

LangChain ofrece separadores especializados para distintos formatos (texto libre, prompts sensibles al recuento de tokens o documentos con encabezados). La siguiente celda crea un pequeño laboratorio para compararlos y anotar en qué situaciones conviene cada uno.


In [78]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

try:
    from langchain_text_splitters import TokenTextSplitter
except ImportError: 
    TokenTextSplitter = None


def explore_text_splitters(raw_text: str):
    markdown_sample = (
        "# Informe de seguridad"
        "## Ataque"
        "### Descripción"
        "Stuxnet combinó múltiples vulnerabilidades de Windows para infiltrarse en las centrifugadoras."
        "### Impacto"
        "Los controladores PLC recibieron cargas manipuladas que modificaron la velocidad de rotación."
    )

    configs = [
        {
            "name": "RecursiveCharacterTextSplitter",
            "description": "Priorizamos saltos de párrafo y frases antes de cortar caracteres sueltos; es ideal para PDF o reportes narrativos.",
            "factory": lambda: RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                separators=["\n\n", "\n", " ", ""],
            ),
            "method": "create_documents",
            "text": raw_text,
        },
    ]

    if TokenTextSplitter is not None:
        configs.append(
            {
                "name": "TokenTextSplitter",
                "description": "Divide por recuento de tokens (útil para modelos con límites estrictos y textos en varios idiomas).",
                "factory": lambda: TokenTextSplitter(chunk_size=256, chunk_overlap=40),
                "method": "create_documents",
                "text": raw_text,
            }
        )
    else:
        print("TokenTextSplitter requiere la dependencia opcional `tiktoken`. Instálala si quieres probarlo.")

    configs.append(
        {
            "name": "MarkdownHeaderTextSplitter",
            "description": "Respeta jerarquías de encabezados Markdown; perfecto para documentación técnica o actas con secciones claras.",
            "factory": lambda: MarkdownHeaderTextSplitter(
                headers_to_split_on=[("#", "titulo"), ("##", "seccion"), ("###", "subseccion")],
            ),
            "method": "split_text",
            "text": markdown_sample,
        }
    )

    for cfg in configs:
        splitter = cfg["factory"]()
        sample_text = cfg["text"]

        try:
            docs = (
                splitter.split_text(sample_text)
                if cfg["method"] == "split_text"
                else splitter.create_documents([sample_text])
            )
        except Exception as exc:
            print(f"{cfg['name']}: no se pudo procesar -> {exc}")
            continue

        lengths = [len(doc.page_content) for doc in docs]
        avg_len = int(sum(lengths) / len(lengths)) if lengths else 0
        snippet = docs[0].page_content[:150].replace("\n", " ") if docs else ""
        print("")
        print(f"{cfg['name']}")
        print(f"  Uso recomendado: {cfg['description']}")
        print(
            f"  documentos: {len(docs)} | media: {avg_len} | min: {min(lengths) if lengths else 0} | max: {max(lengths) if lengths else 0}"
        )
        if docs:
            print(f"  ejemplo: {snippet}...")
            metadata_preview = getattr(docs[0], 'metadata', {})
            if metadata_preview:
                print(f"  metadata ejemplo: {metadata_preview}")


explore_text_splitters(loader)



RecursiveCharacterTextSplitter
  Uso recomendado: Priorizamos saltos de párrafo y frases antes de cortar caracteres sueltos; es ideal para PDF o reportes narrativos.
  documentos: 25 | media: 907 | min: 272 | max: 997
  ejemplo: Seminario  de  Actualización  III:   CiberSeguridad   Exposición:  Casos  Reales  Relacionados  a  la   CiberSeguridad   –   Stuxnet     Grupo:   Quem...

TokenTextSplitter
  Uso recomendado: Divide por recuento de tokens (útil para modelos con límites estrictos y textos en varios idiomas).
  documentos: 45 | media: 509 | min: 385 | max: 1665
  ejemplo: Seminario  de  Actualización  III:   CiberSeguridad   Exposición:  Casos  Reales  Relacionados  a  la   CiberSeguridad   –   Stuxnet     Grupo:   Quem...

MarkdownHeaderTextSplitter
  Uso recomendado: Respeta jerarquías de encabezados Markdown; perfecto para documentación técnica o actas con secciones claras.
  documentos: 0 | media: 0 | min: 0 | max: 0



## Exploración de Modelos de Embedding y LLM
2. Exploración de Modelos de Embedding y LLM:
- Experimenta con diferentes modelos de embedding disponibles en `langchain_ollama` (además de `"nomic-embed-text"`) y analiza cómo impactan en la calidad de la recuperación de documentos.
- Prueba con otros modelos de lenguaje de gran tamaño (LLM) compatibles con `langchain_google_genai` o `langchain_ollama` y compara sus respuestas y rendimiento.

Las celdas siguientes automatizan estas comparativas para que puedas repetirlas fácilmente.



### Comparativa rápida de embeddings
El bloque de código crea embeddings temporales para cada modelo indicado, calcula la similitud coseno contra varias preguntas y muestra los fragmentos más relevantes. Ajusta la lista `candidate_embedding_models`, las preguntas o el parámetro `top_k` según lo necesites.


In [79]:

import numpy as np
from typing import Dict, List
from langchain_core.documents import Document

candidate_embedding_models = [
    "nomic-embed-text",
    "mxbai-embed-large"
]


def _normalize(vector):
    arr = np.array(vector, dtype=float)
    norm = np.linalg.norm(arr)
    return arr if norm == 0 else arr / norm


def compare_embedding_models(model_names: List[str], documents: List[Document], questions: List[str], top_k: int = 3) -> Dict:
    doc_texts = [doc.page_content for doc in documents]
    results = {}

    for model_name in model_names:
        temp_embedding = OllamaEmbeddings(model=model_name)

        doc_vectors = [_normalize(vec) for vec in temp_embedding.embed_documents(doc_texts)]
        results[model_name] = {}

        for question in questions:
            query_vec = _normalize(temp_embedding.embed_query(question))

            scored_docs = sorted(
                (
                    (float(np.dot(query_vec, doc_vec)), doc)
                    for doc_vec, doc in zip(doc_vectors, documents)
                ),
                key=lambda item: item[0],
                reverse=True,
            )[:top_k]

            results[model_name][question] = scored_docs

    return results


def show_embedding_results(results):
    for model_name, question_results in results.items():
        print(f"\nModelo de embedding: {model_name}")

        for question, docs_with_scores in question_results.items():
            print(f" Pregunta: {question}")

            for rank, (score, doc) in enumerate(docs_with_scores, start=1):
                snippet = doc.page_content[:180].replace("\n", " ")
                print(f"   {rank}. score={score:.3f} -> {snippet}...")


example_questions = [
    "¿Qué es Stuxnet?",
    "¿Cómo se propaga?"
]

embedding_comparisons = compare_embedding_models(
    candidate_embedding_models,
    texts,              
    example_questions,
    top_k=2,
)

show_embedding_results(embedding_comparisons)


Modelo de embedding: nomic-embed-text
 Pregunta: ¿Qué es Stuxnet?
   1. score=0.744 -> Introducción   En  el  respectivo  trabajo  vamos  a  abordar  el  caso  de  Stuxnet,  uno  de  los   ciberataques   más   reconocidos   y   con   un   impacto   importante   en   ...
   2. score=0.731 -> en   constante   cambio   (CiberInseguro,   2022).   El  caso  de  Stuxnet  dejó  en  claro  que  las  guerras  modernas  no  solo  se  libran  con   armas   tradicionales,   sino ...
 Pregunta: ¿Cómo se propaga?
   1. score=0.579 -> que   se   desarrolle   e   implemente   un   parche   para   solucionarla.   La   falta   de   un   parche   disponible   representa   un   riesgo   significativo,   ya   que   lo...
   2. score=0.576 -> específicamente   para   atacar   centrifugadoras   Siemens   utilizadas   en   la   planta   nuclear   de   Natanz,   en   Irán,   lo   que   muestra   que   no   se   trataba   d...

Modelo de embedding: mxbai-embed-large
 Pregunta: ¿Qué es Stuxnet?
   1. score=0.87


### Comparativa rápida de LLM
Con esta celda puedes invocar varios modelos de `langchain_google_genai` o `langchain_ollama` con la misma pregunta y contexto recuperado. Modifica `llm_candidates` para incluir los modelos que tengas disponibles en tu entorno.


In [80]:

from typing import Callable, Dict, List, Sequence, Tuple
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")

prompt = """Usa el siguiente contexto para responder:

Contexto:
{contexto}

Pregunta:
{input_user}

Respuesta breve y directa:
"""

def compare_llms(candidates: Sequence[Tuple[str, Callable[[], object]]], question: str, context_docs: List):
    context_text = "\n".join(doc.page_content for doc in context_docs)
    answers = {}

    for name, factory in candidates:
        llm = factory()

       
        completion = llm.invoke(prompt.format(contexto=context_text, input_user=question))

     
        content = getattr(completion, "content", getattr(completion, "text", str(completion)))

        answers[name] = content

    return answers


llm_candidates = [
    ("gemini-2.0-flash", lambda: ChatGoogleGenerativeAI(api_key=api_key, model="gemini-2.0-flash", temperature=0.4)),
    ("qwen2:1.5b", lambda: ChatOllama(model="qwen2:1.5b", temperature=0.2)),
]

sample_question = "Resume brevemente qué problema resuelve este flujo RAG."
context_docs = retrieval(sample_question)

llm_comparisons = compare_llms(llm_candidates, sample_question, context_docs)

for name, answer in llm_comparisons.items(): 
    print(f"\n{name}\n{'-' * len(name)}\n{answer}\n")


gemini-2.0-flash
----------------
El flujo RAG proporciona información sobre el malware Stuxnet, incluyendo su propagación, objetivos, y el uso de vulnerabilidades zero-day.


qwen2:1.5b
----------
El flujo RAG resuelve el problema de la propagación del virus Stuxnet a través de redes informáticas cerradas.



In [81]:
print(format_docs_with_sources(docs))


[Fuente 1 | src=Stuxnet (1) - copia.pdf | chunk=12 | topic=stuxnet | tags=Windows]en
 
constante
 
cambio
 
(CiberInseguro,
 
2022).
 
El  caso  de  Stuxnet  dejó  en  claro  que  las  guerras  modernas  no  solo  se  libran  con  
armas
 
tradicionales,
 
sino
 
también
 
en
 
el
 
terreno
 
digital.
 
Este
 
malware
 
marcó
 
un
 
antes
 
y
 
un
 
después
 
porque
 
demostró
 
que
 
era
 
posible
 
dañar
 
infraestructuras
 
críticas
 
de
 
forma
 
remota
 
y
 
sin
 
intervención
 
directa
 
en
 
el
 
terreno.
Stuxnet  salió  de  Irán  
La  forma  en  que  Stuxnet  salió  de  Irán  fue  mediante  computadoras  de  contratistas,  
memorias
 
USB
 
y
 
redes
 
de
 
empresas
 
multinacionales.
 
Se
 
detectaron
 
infecciones
 
en
 
diferentes
 
países:
 
India  e  Indonesi...
[Fuente 2 | src=Stuxnet (1) - copia.pdf | chunk=2 | topic=stuxnet | tags=sin-tags]Introducción   En  el  respectivo  trabajo  vamos  a  abordar  el  caso  de  Stuxnet,  uno  de  los  
ciberataques
 
más
 
reconocid

Interactuamos con nuestro RAG aplicando filtros opcionales y escogiendo la variante de prompt que mejor se adapte al estilo deseado.


In [82]:
question = input("Pregunta del humano: ").strip()
if not question:
    raise ValueError("Debes ingresar una pregunta para continuar.")

variant_hint = ", ".join(PROMPT_VARIANTS.keys())
chosen_variant = input(
    f"Elige prompt variant ({variant_hint}) [citations]: "
).strip().lower() or "citations"

if chosen_variant not in PROMPT_VARIANTS:
    print("Variante desconocida, usando 'citations'.")
    chosen_variant = "citations"

answer_language = input("Idioma de respuesta [es]: ").strip() or "es"
tone = input("Tono deseado [directa]: ").strip() or "directa"

docs = retrieval(question)
context_block = format_docs_with_sources(docs)

preview = (
    context_block
    if len(context_block) <= 1200
    else context_block[:1200] + "..."
)

print("\nContexto preparado (previsualización):\n")
print(preview)
print("\nRespuesta del LLM:\n")

for chunk in response(
    input_user=question,
    contexto=context_block,
    prompt_variant=chosen_variant,
    answer_language=answer_language,
    style=tone,
):
    print(chunk, end="", flush=True)



Contexto preparado (previsualización):

[Fuente 1 | src=Stuxnet (1) - copia.pdf | chunk=12 | topic=stuxnet | tags=Windows]en
 
constante
 
cambio
 
(CiberInseguro,
 
2022).
 
El  caso  de  Stuxnet  dejó  en  claro  que  las  guerras  modernas  no  solo  se  libran  con  
armas
 
tradicionales,
 
sino
 
también
 
en
 
el
 
terreno
 
digital.
 
Este
 
malware
 
marcó
 
un
 
antes
 
y
 
un
 
después
 
porque
 
demostró
 
que
 
era
 
posible
 
dañar
 
infraestructuras
 
críticas
 
de
 
forma
 
remota
 
y
 
sin
 
intervención
 
directa
 
en
 
el
 
terreno.
Stuxnet  salió  de  Irán  
La  forma  en  que  Stuxnet  salió  de  Irán  fue  mediante  computadoras  de  contratistas,  
memorias
 
USB
 
y
 
redes
 
de
 
empresas
 
multinacionales.
 
Se
 
detectaron
 
infecciones
 
en
 
diferentes
 
países:
 
India  e  Indonesi...
[Fuente 2 | src=Stuxnet (1) - copia.pdf | chunk=2 | topic=stuxnet | tags=sin-tags]Introducción   En  el  respectivo  trabajo  vamos  a  abordar  el  caso  de  Stuxnet,  uno 